In [1]:
# | default_exp classes.DomoDataflow

In [2]:
# | exporti

from __future__ import annotations

from enum import Enum
from dataclasses import dataclass, field
import httpx

import datetime as dt
import domolibrary.utils.convert as ct
import domolibrary.utils.DictDot as util_dd

import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de
import domolibrary.routes.dataflow as dataflow_routes
from nbdev.showdoc import patch_to, show_doc

In [3]:
# | exporti


class DomoDataflow_Action_Type(Enum):
    LoadFromVault = "LoadFromVault"
    PublishToVault = "PublishToVault"
    GenerateTableAction = "GenerateTableAction"


@dataclass
class DomoDataflow_Action:
    id: str
    name: str
    depends_on : str
    type: str

    datasource_id: str  = None
    sql: str = None

    @classmethod
    def _from_json(cls, obj: dict):
        dd = obj

        if isinstance(dd, dict):
            dd = util_dd.DictDot(obj)

        tbl_name = dd.dataSource.name if dd.dataSource else None
        ds_id = dd.dataSource.guid if dd.dataSource else None

        return  cls(
            type=dd.type,
            id=dd.id,
            name=dd.name or dd.targetTableName or dd.tableName or tbl_name,
            depends_on = dd.dependsOn,
            
            datasource_id=dd.dataSourceId or ds_id,
            sql=dd.selectStatement or dd.query,
        )

@dataclass
class DomoDataflow_ActionExecuted(DomoDataflow_Action):
    begin_time : dt.datetime = None
    end_time : dt.datetime = field(init = False )
    is_success : bool = field(init=False)
    rows_process : int = field(init = False)

    @classmethod
    def _from_json(cls, obj):
        dd = obj

        if isinstance(dd, dict):
            dd = util_dd.DictDot(obj)

        base_cls = super()._from_json(obj)

        print(base_cls.__dict__, base_cls.__class__.__name__)

        cls(
            rows_processed =  dd.rowsProcessed,
            is_success = dd.wasSuccessful,
            begin_time = dd.beginTime,
            end_time = dd.endTime,

            **base_cls.__dict__,
        
        )

#### sample implementation of DataFlow_Action

Note: this list of properties that can be attached to `DomoDataflow_Action` is by no means comprehensive. To capture more data from the API, simply add properties to the class and extend the `DomoDataflow_Action._from_json` private method.


In [4]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

dataflow_id = 70
dataflow_id = 185

res = await dataflow_routes.get_dataflow_by_id(dataflow_id=dataflow_id, auth=token_auth)
actions_ls = res.response.get('actions')

[
    DomoDataflow_Action._from_json(action)
    for action in actions_ls
]

[DomoDataflow_Action(id='b9bb0f01-2517-4595-a077-c9778e5ffc48', name='monit_user_accesslist', depends_on=None, type='LoadFromVault', datasource_id='6ddbcb8d-0f38-48ad-bd73-f6b35c4b7daf', sql=None),
 DomoDataflow_Action(id='52f47029-06e0-4599-9ed8-21c4b9386c89', name='Rank & Window', depends_on=['b9bb0f01-2517-4595-a077-c9778e5ffc48'], type='WindowAction', datasource_id=None, sql=None),
 DomoDataflow_Action(id='623883e9-bdc1-44cd-9fb4-6277d0e123de', name='Add Formula', depends_on=['52f47029-06e0-4599-9ed8-21c4b9386c89'], type='ExpressionEvaluator', datasource_id=None, sql=None),
 DomoDataflow_Action(id='99ad5978-9a19-41e1-99d7-3d0ce58f7467', name='Alter Columns', depends_on=['623883e9-bdc1-44cd-9fb4-6277d0e123de'], type='Metadata', datasource_id=None, sql=None),
 DomoDataflow_Action(id='4cf42103-c52d-4b92-b26f-63832ead11ee', name='Select Columns', depends_on=['99ad5978-9a19-41e1-99d7-3d0ce58f7467'], type='SelectValues', datasource_id=None, sql=None),
 DomoDataflow_Action(id='435803f2-08

In [5]:
# | export
@dataclass
class DomoDataflow:
    id: str
    name: str = None
    auth: dmda.DomoAuth = field(default=None)
    owner: str = None
    description: str = None
    tags: list[str] = None
    actions: list[DomoDataflow_Action] = None
    execution_history : list[DataflowExecution_Summary] = None

In [6]:
# | exporti
@patch_to(DomoDataflow, cls_method=True)
async def get_from_id(
    cls: DomoDataflow,
    dataflow_id: int,
    auth: dmda.DomoAuth = None,
    return_raw: bool = False,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
):
    res = await dataflow_routes.get_dataflow_by_id(
        auth=auth,
        dataflow_id=dataflow_id,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=cls.__name__,
        session=session,
    )

    if return_raw:
        return res

    if not res.is_success:
        return None

    dd = util_dd.DictDot(res.response)
    domo_dataflow = cls(
        auth=auth,
        id=dd.id,
        name=dd.name,
        description=dd.description,
        owner=dd.owner,
        tags=dd.tags,
    )

    if dd.actions:
        domo_dataflow.actions = [
            DomoDataflow_Action._from_json(action) for action in dd.actions
        ]

    return domo_dataflow

#### sample implementation of get_from_id


In [7]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await DomoDataflow.get_from_id(dataflow_id=70, auth=token_auth)

DomoDataflow(id=70, name='DomoStats - people - reports', auth=DomoTokenAuth(domo_instance='domo-community', token_name='token_auth', is_valid_token=True, url_manual_login='https://domo-community.domo.com/auth/index?domoManualLogin=true'), owner=None, description=None, tags=None, actions=[DomoDataflow_Action(id='241025d7-3cca-4369-b7c0-b3264277c0e1', name='domostats_people', depends_on=None, type='LoadFromVault', datasource_id='241025d7-3cca-4369-b7c0-b3264277c0e1', sql=None), DomoDataflow_Action(id='c176aa53-122a-4a2e-8b6f-41ca4a72d67c', name='t_1', depends_on=['241025d7-3cca-4369-b7c0-b3264277c0e1'], type='GenerateTableAction', datasource_id=None, sql='SELECT\n`Display Name`,\n`User ID`\n, @row := @row+1 as id\n, @isReset := case when @iter = @rowCounter then 1 else 0 end isReset\n, @group := @isReset + @group   as rowGroup\n, @iter := case when @isReset = 1 then 1 else @iter + 1 end iter\n, @rowCounter := case when @isReset = 1 then FLOOR( RAND() * (7-3) + 3) else @rowCounter end num

In [8]:
@dataclass
class DomoDataflow_ExecutionSummary:
    auth: dmda.DomoAuth = field(repr=False)
    id: str
    dataflow_id: str
    dataflow_execution_id: str
    dataflow_version: str

    begin_time: dt.datetime
    end_time: dt.datetime
    last_updated: dt.datetime

    is_failed: bool
    state: str
    activation_type: str
    data_processor: str
    telemetry: dict
    execution_stats: dict

    @classmethod
    def _from_json(cls, df_obj, auth: dmda.DomoAuth):
        return cls(
            auth=auth,
            id=df_obj["id"],
            dataflow_id=df_obj["onboardFlowId"],
            dataflow_execution_id=df_obj["dapDataFlowExecutionId"],
            dataflow_version=df_obj.get("dataFlowVersion"),
            begin_time=ct.convert_epoch_millisecond_to_datetime(df_obj["beginTime"]),
            end_time=ct.convert_epoch_millisecond_to_datetime(df_obj["endTime"]),
            last_updated=ct.convert_epoch_millisecond_to_datetime(
                df_obj["lastUpdated"]
            ),
            is_failed=df_obj["failed"],
            state=df_obj["state"],
            activation_type=df_obj["activationType"],
            data_processor=df_obj["dataProcessor"],
            telemetry=df_obj.get("telemetry"),
            execution_stats={
                "total_bytes_written": df_obj.get("totalBytesWritten", 0),
                "total_rows_read": df_obj.get("totalRowsRead", 0),
                "total_bytes_read": df_obj.get("totalBytesRead", 0),
                "mean_download_rate_kbps": df_obj.get("meanDownloadRateKbps", 0),
                "total_rows_written": df_obj.get("totalRowsWritten", 0),
            },
        )
        return cls(auth=auth, **df_obj)

In [9]:
# | exporti


@patch_to(DomoDataflow)
async def get_execution_history(
    self: DomoDataflow,
    auth: dmda.DomoAuth = None,
    maximum=100,  # maximum number of execution histories to retrieve
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
    return_raw: bool = False,
):
    """retrieves metadata about execution history.  includes details like execution status"""

    auth = auth or self.auth

    res = await dataflow_routes.get_dataflow_execution_history(
        auth=auth,
        dataflow_id=self.id,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=self.__class__.__name__,
        maximum=maximum,
    )

    if return_raw:
        return res

    self.execution_history = [
        DomoDataflow_ExecutionSummary._from_json(df_obj, auth) for df_obj in res.response
    ]

    return self.execution_history

In [10]:
show_doc(DomoDataflow.get_execution_history)

---

[source](https://github.com/jaewilson07/domo_library/blob/main/domolibrary/classes/DomoDataflow.py#L147){target="_blank" style="float:right; font-size:smaller"}

### DomoDataflow.get_execution_history

>      DomoDataflow.get_execution_history
>                                          (auth:domolibrary.client.DomoAuth.Dom
>                                          oAuth=None, maximum=100,
>                                          debug_api:bool=False,
>                                          debug_num_stacks_to_drop=2,
>                                          return_raw:bool=False)

retrieves metadata about execution history.  includes details like execution status

In [11]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_dataflow = DomoDataflow(id=185, auth=token_auth)

await domo_dataflow.get_execution_history(debug_api=False, return_raw=False)

[DomoDataflow_ExecutionSummary(id=327615, dataflow_id=185, dataflow_execution_id='98fb0606-fa1b-48ea-bf96-0f3a052b7e99', dataflow_version=741, begin_time=datetime.datetime(2023, 6, 15, 21, 17, 58), end_time=datetime.datetime(2023, 6, 15, 21, 18, 9), last_updated=datetime.datetime(2023, 6, 15, 21, 18, 9), is_failed=False, state='SUCCESS', activation_type='MANUAL', data_processor='MAGIC', telemetry=[{'name': 'activeOptimizations', 'value': 'c'}], execution_stats={'total_bytes_written': 2299757, 'total_rows_read': 5988, 'total_bytes_read': 624402, 'mean_download_rate_kbps': 239.78276764648052, 'total_rows_written': 5988}),
 DomoDataflow_ExecutionSummary(id=327600, dataflow_id=185, dataflow_execution_id='33118cae-2a9f-45dc-96dc-7bad72301333', dataflow_version=704, begin_time=datetime.datetime(2023, 6, 15, 21, 3, 5), end_time=datetime.datetime(2023, 6, 15, 21, 3, 16), last_updated=datetime.datetime(2023, 6, 15, 21, 3, 16), is_failed=False, state='SUCCESS', activation_type='MANUAL', data_pro

In [12]:
# | exporti
@patch_to(DomoDataflow)
async def execute(
    self: DomoDataflow,
    auth: dmda.DomoAuth = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
):
    return await dataflow_routes.execute_dataflow(
        auth=auth or self.auth,
        dataflow_id=self.id,
        parent_class=self.__class__.__name__,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
    )

#### sample execute_dataflow


In [13]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_dataflow = DomoDataflow(id=70, auth=token_auth)

await domo_dataflow.execute()

ResponseGetData(status=200, response={'id': 434017, 'onboardFlowId': 70, 'previewRows': 0, 'dapDataFlowExecutionId': '3c548a62-911f-4830-b4b9-debf5af16f06', 'beginTime': 1701837531130, 'lastUpdated': 1701837532426, 'state': 'CREATED', 'activationType': 'MANUAL', 'executionEngine': {'platform': 'K8S', 'engine': 'MYSQL_SQL_ENGINE'}, 'dataProcessor': 'MYSQL', 'dataFlowVersion': 262}, is_success=True, parent_class=None)

In [14]:
# | hide
import nbdev

nbdev.nbdev_export()